## Data Connection 處理流程  
* 文檔載入器(Document loaders): 從多種數據源加載文檔，ex.網頁、pdf。
* 文檔轉換器: 拆分文檔、丟棄冗餘文檔，主要運行在文檔載入器之後，針對加載出來的文檔做處理。
* 文本嵌入(embedding)模型: 將非結構化的文本轉為浮點數列表
* 向量儲存站: 存儲和搜尋 embedding 數據
* 檢索器: 查詢向量數據

![Data_Connection](./image/Data_Connection.JPG)

### 1. 文本載入器(Document loaders)

將文本數據從原始數據(Source)中提取出來，改成 langchain 認識的語言，總而言之就是將非結構化的文本數據加載到結構化的字符串中。  
* 輸入: 各種數據源，ex.PDF、URL、影片。
* 輸出: 一系列的 Document 對象，例，有6頁pdf會產出6個 documents，以分別對應。
    ![Document_loaders.JPG](./image/Document_loaders.JPG)  

In [1]:
from langchain.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path='./data/file.csv')
data = loader.load()

In [2]:
print(data)

[Document(metadata={'source': './data/file.csv', 'row': 0}, page_content='學號: 11127101\n姓名: 小張\n生日: 7月8日'), Document(metadata={'source': './data/file.csv', 'row': 1}, page_content='學號: 11127102\n姓名: 小林\n生日: 6月12日'), Document(metadata={'source': './data/file.csv', 'row': 2}, page_content='學號: 11127103\n姓名: 小雨\n生日: 10月17日')]


In [3]:
len(data)

3

In [4]:
data[0].page_content

'學號: 11127101\n姓名: 小張\n生日: 7月8日'

### 2. 文本分割器(Document transformers)

將加載好的文檔進行轉換，從而更好的適應各種場景。舉例，將文檔拆分成較小的塊，以避免大型語言模型對於輸入長度的限制。  
Langchain 中提供的文檔轉換器可以提供拆分、合併、過濾等功能。

* 文本分割器-拆分: 分割長文本，根據語意相關性將所有有關聯的文本放在同一個分割段中。  
![Document_transformers.JPG.JPG](./image/Document_transformers.JPG)
1. 將文本拆分為小、具語意意義的塊。
2. 將小塊組合成大塊，直到達到一定規模。
3. 將達到一定規模的塊作為獨立的文本片段，然後創建新的文本塊，此外，為了維持塊間的連貫性，兩個文本塊之間會有重疊的部分。  
以圖例而言，Document 由一塊變為三塊。

### 3. 文本詞嵌入(Word Embedding)
詞嵌入是將詞語數值化表達的方式，通常會將詞映射到高維的向量中，使電腦藉由高維的數字化表達得以理解自然語言的語意，接近的語意=接近的向量距離。   


### 4. 向量數據庫
用於儲存嵌入的數據向量。

### 5. 檢索器
根據輸入的非結構化查詢語句返回對應文檔的接口(一系列 Documents 對象)。  
不同於向量數據庫，向量數據庫可以視為一種具備儲存功能的檢索器，但檢索器不一定需要具備儲存的功能。

## 程式事例

### 使用 pdf loader

In [5]:
# 安裝依賴
%pip install pypdf

Note: you may need to restart the kernel to use updated packages.


In [2]:
# 從 langchain 引入 pdf 載入器
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader('./data/PDF_file.pdf')

In [3]:
# 加載 PDF 文檔到 documents 對象
documents = loader.load()

In [8]:
documents

[Document(metadata={'producer': 'AFPL Ghostscript 8.13', 'creator': 'PrimoPDF http://www.primopdf.com', 'creationdate': 'D:20090501171152', 'moddate': 'D:20090501171152', 'title': '(Microsoft Word - LE PETIT PRINCE\\244\\244\\244\\345\\252\\251.doc)', 'author': 'user', 'source': './data/PDF_file.pdf', 'total_pages': 54, 'page': 0, 'page_label': '1'}, page_content='1 \n                         LE PETIT PRINCE \n                             小 王 子 \n                       [ 法] 聖﹒德克旭貝里 \n（此劇本由簡體中文版轉錄而來） \n*************************************************** ****************** \n \n                        獻給列翁﹒維爾特 \n \n    我請孩子們原諒我把這本書獻給了一個大人。我有一個很重要的理由：這個 \n大人是我在世界上最好的朋友。我還有另一個理由：這個大人他什么都能懂，甚 \n至給孩子們寫的書他也能懂。我的第三個理由是：這個大人住在法國，他在那里 \n挨餓、受凍。他很需要安慰。如果這些理由還不夠的話，那么我愿意把這本書獻 \n給兒童時代的這個大人。所有的大人都曾經是孩子。 （可惜，只有很少的一些大 \n人記得這一點。）因此，我就把獻詞改為： \n \n                  獻給還是小男孩時的列翁﹒維爾特 \n \n*************************************************** ****************** \n \n                         LE PETIT PRINCE \n   

In [9]:
# 共54頁，每頁 pdf 對應一個 Document 對象
len(documents)

54

In [10]:
documents[10]

Document(metadata={'producer': 'AFPL Ghostscript 8.13', 'creator': 'PrimoPDF http://www.primopdf.com', 'creationdate': 'D:20090501171152', 'moddate': 'D:20090501171152', 'title': '(Microsoft Word - LE PETIT PRINCE\\244\\244\\244\\345\\252\\251.doc)', 'author': 'user', 'source': './data/PDF_file.pdf', 'total_pages': 54, 'page': 10, 'page_label': '11'}, page_content='11  \n \n    小王子一旦提出了問題，從來不會放過。這個該死的螺絲使我很惱火，我于 \n是就隨便回答了他一句： \n \n    “刺么，什么用都沒有，這純粹是花的惡劣表現。” \n \n    “噢！” \n \n    可是他沉默了一會兒之后，懷著不滿的心情沖我說： \n \n    “我不信！花是弱小的、淳朴的，它們總是設法保護自己，以為有了刺就可 \n以顯出自己的厲害…” \n \n    我默不作聲。我當時想的，如果這個螺絲再和我作對，我就一錘子敲掉它。 \n小王子又來打攪我的思緒了： \n \n    “你卻認為花…” \n \n    “算了吧，算了吧！我什么也不認為！我是隨便回答你的。我可有正經事要 \n做。” \n \n    他驚訝地看著我。 \n \n    “正經事？” \n \n    他瞅著我手拿錘子， 手指沾滿了油污， 伏在一個在他看來丑不可言的機件上。 \n \n    “你說話就和那些大人一樣！” \n \n    這話使我有點難堪。可是他又尖刻無情地說道： \n \n    “你什么都分不清…你把什么都混在一起！” \n \n    他著實非常惱火。搖動著腦袋，金黃色的頭發隨風顫動著。 \n \n    “我到過一個星球，上面住著一個紅臉先生。他從來沒聞過一朵花。他從來 \n沒有看過一顆星星。 他什么人也沒有喜歡過。 除了算帳以外， 他什么也沒有做過。 \n他整天同你一樣老是說：‘我有正經事，我是個

### 文本分割器: RecursiveCharacterTestSplitter



In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    # set the chunk size
    chunk_size=100, # 每個 chunk 的最大字元數
    chunk_overlap=20, # 每個 chunk 之間的重疊字元數
    length_function=len # 用於計算字元長度的函數，默認為 len
)

In [5]:
# 從 langchain 引入 pdf 載入器
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader('./data/PDF_file.pdf')

# load_and_split 可在加載時提供文本分割器
pages = loader.load_and_split(text_splitter=text_splitter)  

In [13]:
len(pages)

472

In [14]:
# 類型仍是 Document 對象，但 page_content 中的內容已經被分割成更小的 chunk
pages[100]

Document(metadata={'producer': 'AFPL Ghostscript 8.13', 'creator': 'PrimoPDF http://www.primopdf.com', 'creationdate': 'D:20090501171152', 'moddate': 'D:20090501171152', 'title': '(Microsoft Word - LE PETIT PRINCE\\244\\244\\244\\345\\252\\251.doc)', 'author': 'user', 'source': './data/PDF_file.pdf', 'total_pages': 54, 'page': 9, 'page_label': '10'}, page_content='很長時間以后，得出了什么結果一樣，他突然沒頭沒腦地問我： \n \n    “羊，要是吃小灌木，它也要吃花羅？” \n \n    “它碰到什么吃什么。” \n \n    “連有刺的花也吃嗎？”')

### 文本詞嵌入

In [6]:
from langchain_ollama import OllamaEmbeddings

# 使用 ollama 的 embedding 模型
embeddings_model = OllamaEmbeddings(model="llama3.1:8b")

'''
# 也可以使用 OpenAI 的 embedding 模型，但需要配置 OpenAI API 密鑰
# 需要在環境變量中設置 OPENAI_API_KEY
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()
'''

'\n# 也可以使用 OpenAI 的 embedding 模型，但需要配置 OpenAI API 密鑰\n# 需要在環境變量中設置 OPENAI_API_KEY\nfrom langchain.embeddings import OpenAIEmbeddings\nembeddings = OpenAIEmbeddings()\n'

In [7]:
# 將第100頁的內容轉換為詞嵌入
embeddings = embeddings_model.embed_documents([pages[100].page_content])


In [17]:
len(embeddings), len(embeddings[0])

# 只傳入1頁，因此 embeddings 只包含一個詞嵌入
# 而在這頁的內容中，詞嵌入的維度為 4096

(1, 4096)

In [18]:
embeddings[0]

[0.0019216958,
 0.009884738,
 0.012358592,
 -0.008736268,
 -0.010747481,
 -0.015540097,
 -0.030026697,
 0.02018724,
 0.02294892,
 0.0069631953,
 -0.004387997,
 -0.00038075546,
 0.003217879,
 0.012226894,
 -0.0031028818,
 0.016207106,
 -0.004277446,
 -0.0155120455,
 0.0015091837,
 -0.0052660736,
 -0.0014875269,
 -0.017990094,
 0.008522435,
 0.00225567,
 -0.032378867,
 0.020270672,
 -0.01378724,
 0.023014812,
 -0.032358304,
 -0.017034417,
 0.013749805,
 0.012385191,
 -0.003897393,
 -0.0047881766,
 -0.00028872464,
 -0.010483781,
 0.0069015925,
 -0.0043464806,
 0.0003132366,
 -0.0129353115,
 -0.01764228,
 0.013196634,
 0.011964831,
 0.007424923,
 -0.006267686,
 0.007899953,
 -0.02507635,
 0.018671036,
 -0.002450823,
 -0.0011920761,
 0.0043888493,
 -0.0032770284,
 -0.003917859,
 -0.0019096149,
 -0.0065176752,
 0.009940115,
 0.034346227,
 0.009729095,
 0.009288223,
 0.010496671,
 -0.0024921743,
 -0.012883552,
 -0.015786488,
 -0.01677606,
 0.010758235,
 -0.025816616,
 0.015324944,
 0.01171463

In [ ]:
test = embeddings_model.embed_documents([page.page_content for page in pages])

In [ ]:
len(test), len(test[0])

(472, 4096)

### 向量儲存

In [ ]:
# 安裝本地向量儲存庫 Chroma
%pip install chromadb
%pip install -qU langchain-chroma

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached markdown_it_py-3.0.0-py3-none-any.whl.metadata (6.9 kB)
  Using cached mdurl-0.1.2-py3-none-any.whl.metadata (1.6 kB)
  Using cached pyasn1-0.6.1-py3-none-any.whl.metadata (8.4 kB)
   ---------------------------------------- 0.0/19.5 MB ? eta -:--:--
   ----- ---------------------------------- 2.9/19.5 MB 18.6 MB/s eta 0:00:01
   ------------ --------------------------- 6.0/19.5 MB 15.4 MB/s eta 0:00:01
   -------------------------- ------------- 12.8/19.5 MB 21.8 MB/s eta 0:00:01
   ---------------------------------------  19.4/19.5 MB 25.0 MB/s eta 0:00:01
   ---------------------------------------- 19.5/19.5 MB 23.7 MB/s eta 0:00:00
   --------


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
# from langchain.embeddings import OllamaEmbeddings
# from langchain.vectorstores import Chroma
from langchain_ollama import OllamaEmbeddings
from langchain_chroma import Chroma

# db 即是向量儲存庫，可用於相似度檢索
db = Chroma.from_documents(pages, embeddings_model)


: 

#### 相似度檢索

In [ ]:
query = "狐狸"

# k=3 表示返回最相似的3個文檔
docs = db.similarity_search(query, k=3)

In [ ]:
len(docs)

In [ ]:
docs[0]

### 檢索器

還未實做 code